In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
from missforest import MissForest
warnings.simplefilter("ignore")

In [33]:
df = pd.read_excel("../../data/restated/Test-Truncated-Restated.xlsx")
normal_df = df.drop(
    ["Completion Date", "Match Support Contact Notes"],
    axis=1
)
contact_note_df = df[['Match ID 18Char', "Completion Date", "Match Support Contact Notes"]]


In [34]:
df.shape

(2566, 63)

In [35]:
# columns to be modified
# todo: remove all "contact" and use _
# these columns are either not relevant for match length or missing enormous amount of values (might use then for eda)
to_be_deleted = [
    "Big Employer/School Census Block Group", "Big Enrollment: Created Date", "Big Acceptance Date",
    "Big Contact: Created Date","Big Days Acceptance to Match", "Big Days Interview to Acceptance","Big Days Interview to Match",
    "Big Contact: Preferred Communication Type", "Big Assessment Uploaded", "Big Enrollment: Created Date",
    "Big Employer", "Big Employer/School Census Block Group", "Big Approved Date", "Big Home Census Block Group", "Big Enrollment: Record Type",

    "Big Contact: Interest Finder - Entertainment","Big Contact: Interest Finder - Hobbies","Big Contact: Interest Finder - Places To Go",
    "Big Contact: Interest Finder - Sports","Little Contact: Interest Finder - Arts","Little Contact: Interest Finder - Career",
    "Little Contact: Interest Finder - Entertainment", "Little Contact: Interest Finder - Hobbies", "Little Contact: Interest Finder - Other Interests",
    "Little Contact: Interest Finder - Outdoors","Little Contact: Interest Finder - Personality","Little Contact: Interest Finder - Places To Go",
    "Little Contact: Interest Finder - Sports","Little Contact: Interest Finder - Three Wishes","Little Other Interests",
    "Little Contact: Language(s) Spoken", "Big Contact: Former Big/Little", "Big Level of Education", "Big: Military",
    "Big Languages", "Big Car Access", "Big Open to Cross-Gender Match", "Big Contact: Volunteer Availability", "Big Contact: Marital Status", "Big Re-Enroll", "Big County",
                    

    "Big ID", "Little ID", "Big Assessment Uploaded", 
    "Little Mailing Address Census Block Group", "Little Interview Date", "Little Acceptance Date", "Little Application Received", "Little Moved to RTBM in MF",
    "Little RTBM Date in MF", "Little RTBM in Matchforce", "Little Moved to RTBM in MF", "Little Interview Date", "Little Acceptance Date", "Little RTBM in Matchforce",
]

yes_no = [
    "Big Open to Cross-Gender Match", "Big Re-Enroll", "Big Contact: Marital Status", "Big Gender", "Big Military", "Big Car Access",
    "Big Contact: Former Big/Little", 
] 

In [36]:
normal_df.drop(to_be_deleted, axis=1, inplace=True)

In [37]:
# convert all column to lower case
normal_df.columns = (
    normal_df.columns.str.strip()  # Remove leading/trailing spaces
               .str.replace(':', '_', regex=True)  # Replace colons (`:`) with underscores
               .str.replace(r'\s+', '_', regex=True)  # Replace any spaces with underscores
)
normal_df.columns = [
    x.lower().\
        replace("contact: ", "").\
        replace("finder - ", "").\
        replace("/","_").\
        replace("_18char","").\
        replace(" ", "_")
    for x in normal_df.columns]
normal_df.columns

Index(['match_id', 'big_age', 'big_occupation', 'big_gender', 'big_birthdate',
       'program', 'program_type', 'big_race_ethnicity',
       'match_activation_date', 'rationale_for_match', 'little_gender',
       'little_participant__race_ethnicity', 'little_birthdate'],
      dtype='object')

In [38]:
# Identify categorical and datetime columns
categorical_cols = normal_df.select_dtypes(include=['object']).columns
datetime_cols = normal_df.select_dtypes(include=['datetime64']).columns

# Store the datetime column separately
datetime_normal_df = normal_df[datetime_cols]

# Drop the datetime column before imputation
normal_df = normal_df.drop(columns=datetime_cols)

# Convert categorical columns to 'category' dtype
for col in categorical_cols:
    normal_df[col] = normal_df[col].astype('category')

# Save category mappings for conversion after imputation
category_mappings = {col: dict(enumerate(normal_df[col].cat.categories)) for col in categorical_cols}

# Convert categorical columns to numerical codes
for col in categorical_cols:
    normal_df[col] = normal_df[col].cat.codes
    normal_df[col].replace(-1, np.nan, inplace=True)  # Keep missing values as NaN

# Initialize MissForest imputer
imputer = MissForest()

# Perform imputation
imputed_normal_df = imputer.fit_transform(normal_df)

# Convert back to normal_dfFrame
imputed_normal_df = pd.DataFrame(imputed_normal_df, columns=normal_df.columns)

# Convert categorical columns back to original categories
for col in categorical_cols:
    imputed_normal_df[col] = imputed_normal_df[col].round().astype(int)
    imputed_normal_df[col] = imputed_normal_df[col].map(category_mappings[col])

# Add the datetime column back
imputed_normal_df = pd.concat([imputed_normal_df, datetime_normal_df.reset_index(drop=True)], axis=1)

100%|██████████| 2/2 [00:00<00:00, 46.61it/s]


In [39]:
imputed_normal_df['big_race_ethnicity'].dropna(inplace=True, axis=0)
imputed_normal_df.shape

(2566, 13)

In [49]:
imputed_normal_df = imputed_normal_df.groupby("match_id").first()

In [50]:

contact_note_df.rename({
    'Match ID 18Char': "match_id",
    'Completion Date':'completion_date',
    "Match Support Contact Notes":"contact_notes"
},axis=1, inplace=True)
contact_note_df.head()

,match_id,completion_date,contact_notes
0,a1v2J0000027CXKQA2,2017-12-20,Question: Activities: Answer: BB and...
1,a1v2J0000027CXKQA2,2018-01-19,Question: Activities: Answer: BB and...
3,a1v2J0000027JFCQA2,2018-12-05,Question: Activities: Answer: MEC me...
4,a1v2J0000027JFCQA2,2019-02-14,Question: Activities: Answer: MEC sp...
5,a1v2J0000027JFCQA2,2019-05-21,MEC started the conversation by asking how thi...


In [51]:
contact_note_df.dropna(axis=0, inplace=True)

In [52]:
contact_note_df.shape

(2465, 3)

In [53]:
join_df = pd.merge(contact_note_df, imputed_normal_df, how='left', on='match_id')

In [54]:
join_df.isna().sum()

match_id                              0
completion_date                       0
contact_notes                         0
big_age                               0
big_occupation                        0
big_gender                            0
program                               0
program_type                          0
big_race_ethnicity                    0
rationale_for_match                   0
little_gender                         0
little_participant__race_ethnicity    0
big_birthdate                         0
match_activation_date                 0
little_birthdate                      0
dtype: int64

In [55]:
join_df.shape

(2465, 15)

In [56]:
join_df.to_excel("data/cleaning.xlsx", index=False)